# Enhancing RAG with Contextual Retrieval

We will use an LLM to generate for each chunk and document a contextual sentence to improve its retrival accuracy and use in hybrid search.

* Generate the context sentence.
* Enrich the chunk embedding vectors with the context.
* Create a topic database to be used in hybrid search.
* Perform hybrid search to improve retrieval results.

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich) to make the output more readable, and supress warning messages.

In [2]:
from rich.pretty import pprint
from rich.theme import Theme
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

custom_theme = Theme({
    "repr.own": "bright_yellow",            # Class names
    "repr.tag_name": "bright_yellow",       # Adjust tag names which might still be purple
    "repr.call": "bright_yellow",           # Function calls and other symbols
    "repr.str": "bright_green",             # String representation
    "repr.number": "bright_red",            # Numbers
    "repr.attrib_name": "bright_yellow",    # Attribute names
    "repr.attrib_value": "bright_blue"      # Attribute values
})

# Apply the theme and print the object with rich formatting

console = Console(theme=custom_theme)

In [3]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Loading a complex dataset of documents

We will load a complex dataset of scientific documents from Arxiv. Applying naive chunks on such documents will give poor results in RAG applications.

In [4]:
from datasets import load_dataset

dataset = load_dataset("jamescalam/ai-arxiv2", split="train")
console.print(dataset)

Dataset({
    features: ['id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 
'primary_category', 'published', 'updated', 'content', 'references'],
    num_rows: 2673
})

In [5]:
import os
from semantic_router.encoders import OpenAIEncoder

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

encoder = OpenAIEncoder(name="text-embedding-3-small")

In [6]:
from semantic_chunkers import StatisticalChunker
import logging

logging.disable(logging.CRITICAL)

chunker = StatisticalChunker(
    encoder=encoder,
    min_split_tokens=100,
    max_split_tokens=500,
)

In [7]:
chunks_0 = chunker(docs=[dataset["content"][0]])


In [11]:
first_chunk = ' '.join(chunks_0[0][0].splits)
console.print(first_chunk)

4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. Jiang, Alexandre 
Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, Diego de las 
Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio Renard Lavaud, 
Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon Antoniak, Teven Le 
Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed Abstract We introduce 
Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same architecture as Mistral 7B, 
with the difference that each layer is composed of 8 feedforward blocks (i.e. experts). For every token, at each 
layer, a router network selects two experts to process the current state and combine their outputs. Even though 
each token only sees two experts, the selected experts can be different at each timestep. As a result, each token 
has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was trained with a 
context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all evaluated benchmarks. 
In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and multilingual benchmarks.
We also provide a model fine- tuned to follow instructions, Mixtral 8x7B â Instruct, that surpasses GPT-3.5 Turbo, 
Claude-2.1, Gemini Pro, and Llama 2 70B â

In [18]:
from dotenv import load_dotenv

load_dotenv()

True

In [19]:
import anthropic

client = anthropic.Anthropic(
    # This is the default and can be omitted
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)


In [20]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
Here is the chunk we want to situate within the whole document
<chunk>
{chunk_content}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk.
Answer only with the succinct context and nothing else.
"""

def situate_context(doc: str, chunk: str) -> str:
    response = client.beta.prompt_caching.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        temperature=0.0,
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc),
                        "cache_control": {"type": "ephemeral"} #we will make use of prompt caching for the full documents
                    },
                    {
                        "type": "text",
                        "text": CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk),
                    }
                ]
            }
        ],
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )
    return response

In [23]:
chunk_context = situate_context(dataset["content"][0], first_chunk)

In [24]:
console.print(chunk_context)

PromptCachingBetaMessage(
    id='msg_01U87paNMV7Q5odMtg4LpWD8',
    content=[
        TextBlock(
            text='This chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms 
Llama 2 70B and GPT-3.5 on various benchmarks. It also describes the model architecture and the fine-tuned Mixtral 
8x7B - Instruct model.',
            type='text'
        )
    ],
    model='claude-3-haiku-20240307',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=PromptCachingBetaUsage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=12532,
        input_tokens=584,
        output_tokens=73
    )
)

In [25]:
second_chunk = ' '.join(chunks_0[0][1].splits)

In [26]:
second_chunk_context = situate_context(dataset["content"][0], second_chunk)

In [27]:
console.print(second_chunk_context)

PromptCachingBetaMessage(
    id='msg_01WgBrxDRVLdrPoPCTbWsiyt',
    content=[
        TextBlock(
            text='This chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms 
Llama 2 70B and GPT-3.5 on most benchmarks. It describes the key architectural details of Mixtral, including its 
use of a sparse mixture-of-experts network, and mentions that the base and instruct models are released under the 
Apache 2.0 license.',
            type='text'
        )
    ],
    model='claude-3-haiku-20240307',
    role='assistant',
    stop_reason='end_turn',
    stop_sequence=None,
    type='message',
    usage=PromptCachingBetaUsage(
        cache_creation_input_tokens=0,
        cache_read_input_tokens=12532,
        input_tokens=290,
        output_tokens=92
    )
)

In [61]:
arxiv_id = dataset[0]["id"]
refs = list(dataset[0]["references"].values())
doc_text = dataset[0]["content"]
title = dataset[0]["title"]

from tqdm import tqdm

corpus_json = []
for i, chunk in tqdm(enumerate(chunks_0[0]), total=len(chunks_0[0]), desc="Processing chunks"):
    chunk_text = ' '.join(chunk.splits)
    contextualized_text = situate_context(doc_text, chunk_text).content[0].text
    corpus_json.append({
        "id": i,
        "text": f"{chunk_text}\n\n{contextualized_text}",
        "metadata" : {
            "title": title,
            "arxiv_id": arxiv_id,
            "references": refs
        }
    })

Processing chunks: 100%|██████████| 46/46 [10:40<00:00, 13.91s/it]


In [62]:
console.print(corpus_json[:2])

[
    {
        'id': 0,
        'text': '4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # Mixtral of Experts Albert Q. 
Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour, Guillaume Lample, LÃ©lio 
Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon 
Antoniak, Teven Le Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e Lacroix, William El Sayed 
Abstract We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. Mixtral has the same 
architecture as Mistral 7B, with the difference that each layer is composed of 8 feedforward blocks (i.e. experts).
For every token, at each layer, a router network selects two experts to process the current state and combine their
outputs. Even though each token only sees two experts, the selected experts can be different at each timestep. As a
result, each token has access to 47B parameters, but only uses 13B active parameters during inference. Mixtral was 
trained with a context size of 32k tokens and it outperforms or matches Llama 2 70B and GPT-3.5 across all 
evaluated benchmarks. In particular, Mixtral vastly outperforms Llama 2 70B on mathematics, code generation, and 
multilingual benchmarks. We also provide a model fine- tuned to follow instructions, Mixtral 8x7B â Instruct, that 
surpasses GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B â\n\nThis chunk introduces Mixtral 8x7B, a sparse 
mixture of experts language model that outperforms Llama 2 70B and GPT-3.5 on various benchmarks. It also describes
the model architecture and the fine-tuned Mixtral 8x7B - Instruct model.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    {
        'id': 1,
        'text': 'chat model on human bench- marks. Both the base and instruct models are released under the Apache 
2.0 license. Code: https://github.com/mistralai/mistral-src Webpage: https://mistral.ai/news/mixtral-of-experts/ # 
Introduction In this paper, we present Mixtral 8x7B, a sparse mixture of experts model (SMoE) with open weights, 
licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 on most benchmarks. As it only uses a subset
of its parameters for every token, Mixtral allows faster inference speed at low batch-sizes, and higher throughput 
at large batch-sizes. Mixtral is a sparse mixture-of-experts network. It is a decoder-only model where the 
feedforward block picks from a set of 8 distinct groups of parameters. At every layer, for every token, a router 
network chooses two of these groups (the â\n\nThis chunk introduces Mixtral 8x7B, a sparse mixture of experts 
language model that outperforms Llama 2 70B and GPT-3.5 on most benchmarks. It describes the key architectural 
details of Mixtral, including its use of a sparse mixture-of-experts network, and mentions that the base and 
instruct models are released under the Apache 2.0 license.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    }
]

## Hybrid Search

We will use bm25 supported database to complement the semantic search with the vector database.

In [35]:
import bm25s

In [63]:
corpus_text = [doc["text"] for doc in corpus_json]

# Tokenize the corpus and only keep the ids (faster and saves memory)
corpus_tokens = bm25s.tokenize(corpus_text, stopwords="en")

# Create the BM25 retriever and attach your corpus_json to it
retriever = bm25s.BM25(corpus=corpus_json)
# Now, index the corpus_tokens (the corpus_json is not used yet)
retriever.index(corpus_tokens)




Split strings:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/46 [00:00<?, ?it/s]

In [88]:
# Query the corpus
query = "What is context size of Mixtral?"
query_tokens = bm25s.tokenize(query)


results, scores = retriever.retrieve(query_tokens, k=10)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    console.print(f"Rank {i+1} (score: {score:.2f}): {doc}")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 2.11): {'id': 14, 'text': "Active Params French Arc-c HellaS MMLU German Arc-c HellaS MMLU Spanish 
Arc-c HellaS MMLU Italian Arc-c HellaS MMLU 33B 70B 13B 42.9% 65.4% 49.0% 39.3% 68.1% 49.9% 49.9% 72.5% 64.3% 49.4%
70.9% 65.1% 58.2% 77.4% 70.9% 54.3% 73.0% 71.5% 55.4% 77.6% 72.5% 52.8% 75.1% 70.9% 41.1% 63.3% 48.7% 47.3% 68.7% 
64.2% 45.7% 69.8% 52.3% 50.5% 74.5% 66.0% Table 4: Comparison of Mixtral with Llama on Multilingual Benchmarks. On 
ARC Challenge, Hellaswag, and MMLU, Mixtral outperforms Llama 2 70B on 4 languages: French, German, Spanish, and 
Italian. # 3.2 Long range performance To assess the capabilities of Mixtral to tackle long context, we evaluate it 
on the passkey retrieval task introduced in [23], a synthetic task designed to measure the ability of the model to 
retrieve a passkey inserted randomly in a long prompt. Results in Figure 4 (Left) show that Mixtral achieves a 100%
retrieval accuracy regardless of the context length or the position of passkey in the sequence. Figure 4 (Right) 
shows that the perplexity of Mixtral on a subset of the proof-pile dataset [2] decreases monotonically as the size 
of the context increases. Passkey Performance ry 0.8 0.6 04 0.2 0.0 OK 4K 8K 12K 16K 20K 24K 28K Seq Len Passkey 
Loc\n\nThe chunk discusses Mixtral's performance on multilingual benchmarks and its ability to handle long-range 
context, demonstrating its strong capabilities in these areas.", 'metadata': {'title': 'Mixtral of Experts', 
'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 2 (score: 2.11): {'id': 2, 'text': 'expertsâ ) to process the token and combine their output additively. This 
technique increases the number of parameters of a model while controlling cost and latency, as the model only uses 
a fraction of the total set of parameters per token. Mixtral is pretrained with multilingual data using a context 
size of 32k tokens. It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several 
benchmarks. In particular, Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk 
describes the key architectural details of the Mixtral model, a sparse mixture-of-experts language model that 
outperforms larger models like Llama 2 70B and GPT-3.5 on various benchmarks.', 'metadata': {'title': 'Mixtral of 
Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 3 (score: 1.46): {'id': 0, 'text': '4 2 0 2 n a J 8 ] G L . s c [ 1 v 8 8 0 4 0 . 1 0 4 2 : v i X r a # 
Mixtral of Experts Albert Q. Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch, Blanche Savary, Chris 
Bamford, Devendra Singh Chaplot, Diego de las Casas, Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume 
Bour, Guillaume Lample, LÃ©lio Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux, Pierre Stock, Sandeep 
Subramanian, Sophia Yang, Szymon Antoniak, Teven Le Scao, ThÃ©ophile Gervet, Thibaut Lavril, Thomas Wang, TimothÃ©e
Lacroix, William El Sayed Abstract We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language model. 
Mixtral has the same architecture as Mistral 7B, with the difference that each layer is composed of 8 feedforward 
blocks (i.e. experts). For every token, at each layer, a router network selects two experts to process the current 
state and combine their outputs. Even though each token only sees two experts, the selected experts can be 
different at each timestep. As a result, each token has access to 47B parameters, but only uses 13B active 
parameters during inference. Mixtral was trained with a context size of 32k tokens and it outperforms or matches 
Llama 2 70B and GPT-3.5 across all evaluated benchmarks. In particular, Mixtral vastly outperforms Llama 2 70B on 
mathematics, code generation, and multilingual benchmarks. We also provide a model fine- tuned to follow 
instructions, Mixtral 8x7B â Instruct, that surpasses GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B 
â\n\nThis chunk introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms Llama 2 70B 
and GPT-3.5 on various benchmarks. It also describes the model architecture and the fine-tuned Mixtral 8x7B - 
Instruct model.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': 
['1905.07830']}}

...

Rank 6 (score: 1.16): {'id': 4, 'text': "Instruct under the Apache 2.0 license1, free for academic and commercial 
usage, ensuring broad accessibility and potential for diverse applications. To enable the community to run Mixtral 
with a fully open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels 
for efficient inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 
Architectural details Mixtral is based on a transformer architecture [31] and uses the same modifications as 
described in [18], with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, 
and the feed- forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture 
parameters are summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language 
model, including its use of a transformer architecture with a 32k token context length and mixture-of-expert 
layers. It also mentions the model's open-source licensing and deployment options.", 'metadata': {'title': 'Mixtral
of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 7 (score: 0.89): {'id': 41, 'text': 'Hellaswag: Can a machine really finish your sentence? arXiv preprint 
arXiv:1905.07830, 2019. [33] Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang,
Zi Lin, Zhuohan Li, Dacheng Li, Eric Xing, et al. Judging llm-as-a-judge with mt-bench and chatbot arena. arXiv 
preprint arXiv:2306.05685, 2023.\n\nThe chunk discusses two references related to language model benchmarking, 
including the Hellaswag dataset and the MT-Bench and Chatbot Arena benchmarks. This is situated within the broader 
context of the paper, which introduces the Mixtral language model and evaluates its performance on various 
benchmarks.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 8 (score: 0.82): {'id': 3, 'text': 'Figure 1: Mixture of Experts Layer. Each input vector is assigned to 2 of 
the 8 experts by a router. The layerâ s output is the weighted sum of the outputs of the two selected experts. In 
Mixtral, an expert is a standard feedforward block as in a vanilla transformer architecture. Mixtral demonstrates 
superior capabilities in mathematics, code generation, and tasks that require multilingual understanding, 
significantly outperforming Llama 2 70B in these domains. Experiments show that Mixtral is able to successfully 
retrieve information from its context window of 32k tokens, regardless of the sequence length and the location of 
the information in the sequence. We also present Mixtral 8x7B â Instruct, a chat model fine-tuned to follow 
instructions using supervised fine-tuning and Direct Preference Optimization [25]. Its performance notably 
surpasses that of GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B â chat model on human evaluation 
benchmarks. Mixtral â Instruct also demonstrates reduced biases, and a more balanced sentiment profile in 
benchmarks such as BBQ, and BOLD. We release both Mixtral 8x7B and Mixtral 8x7B â\n\nThis chunk describes the 
Mixture of Experts layer architecture used in the Mixtral model, as well as the superior performance of Mixtral 
compared to other models on various benchmarks, including mathematics, code generation, and multilingual tasks. It 
also introduces the Mixtral 8x7B - Instruct model, which is fine-tuned to follow instructions and outperforms other
chat models on human evaluation benchmarks.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088',
'references': ['1905.07830']}}

Rank 9 (score: 0.77): {'id': 30, 'text': 'Quac: Question answering in context. arXiv preprint arXiv:1808.07036, 
2018. [6] Aidan Clark, Diego De Las Casas, Aurelia Guy, Arthur Mensch, Michela Paganini, Jordan Hoffmann, Bogdan 
Damoc, Blake Hechtman, Trevor Cai, Sebastian Borgeaud, et al. Unified scaling laws for routed language models. In 
International Conference on Machine Learning, pages 4057â 4086. PMLR, 2022. [7] Christopher Clark, Kenton Lee, 
Ming-Wei Chang, Tom Kwiatkowski, Michael Collins, and Kristina Toutanova.\n\nThe chunk discusses references related
to question answering and language models, which are relevant topics covered in the overall document.', 'metadata':
{'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 10 (score: 0.68): {'id': 37, 'text': 'Landmark attention: Random-access infinite context length for 
transformers. arXiv preprint arXiv:2305.16300, 2023. [24] Alicia Parrish, Angelica Chen, Nikita Nangia, Vishakh 
Padmakumar, Jason Phang, Jana Thomp- son, Phu Mon Htut, and Samuel R Bowman. Bbq: A hand-built bias benchmark for 
question answering. arXiv preprint arXiv:2110.08193, 2021. [25] Rafael Rafailov, Archit Sharma, Eric Mitchell, 
Stefano Ermon, Christopher D Manning, and Chelsea Finn. Direct preference optimization: Your language model is 
secretly a reward model. arXiv preprint arXiv:2305.18290, 2023. [26] Keisuke Sakaguchi, Ronan Le Bras, Chandra 
Bhagavatula, and Yejin Choi.\n\nThe chunk discusses references related to long-range performance, bias benchmarks, 
and instruction fine-tuning of language models, which are topics covered in the overall document.', 'metadata': 
{'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

In [65]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer

qdrant_client = QdrantClient(
    ":memory:"
) 

# Create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [66]:
collection_name = "hybrid_search"

first_collection = qdrant_client.recreate_collection(
    collection_name=collection_name,
        vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)
print(first_collection)

True


In [67]:
# vectorize!
qdrant_client.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["text"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(corpus_json) # data is the variable holding all the wines
    ]
)

In [68]:
query_vector = encoder.encode(query).tolist()

In [89]:
hits = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10
)

In [90]:
hits

[ScoredPoint(id=15, version=0, score=0.6180975806351034, payload={'id': 15, 'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input sequence. (Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on the proof-pile dataset decreases as the context length increases.", 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', '

In [91]:
documents_with_scores = []
for hit in hits:
    doc_id = hit.payload["id"]
    doc_text = next((doc for doc in corpus_json if doc["id"] == doc_id), None)["text"]
    doc_dense_score = hit.score
    documents_with_scores.append({
        "id": doc_id,
        "text": doc_text,
        "dense_score": doc_dense_score
    })

for i, result in enumerate(results[0]):
    doc_id = result["id"]
    doc_text = next((doc for doc in corpus_json if doc["id"] == doc_id), None)["text"]
    doc_sparse_score = scores[0][i]
    for doc in documents_with_scores:
        if doc["id"] == doc_id:
            doc["sparse_score"] = doc_sparse_score
            break




In [92]:
console.print(documents_with_scores)

[
    {
        'id': 15,
        'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey 
Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 
5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% 
retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input sequence. 
(Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length 
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
        'dense_score': 0.6180975806351034,
        'sparse_score': 1.4473938
    },
    {
        'id': 4,
        'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad 
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
        'dense_score': 0.4972174713599332,
        'sparse_score': 1.1622587
    },
    {
        'id': 2,
        'text': 'expertsâ ) to process the token and combine their output additively. This technique increases the 
number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the total 
set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens. It 
either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
        'dense_score': 0.44239135249907124,
        'sparse_score': 2.1099076
    },
    {
        'id': 6,
        'text': 'Table 1: Model architecture. # j nâ G(x)i Â· Ei(x). i=0 Here, G(x)i denotes the n-dimensional 
output of the gating network for the i-th expert, and Ei(x) is the output of the i-th expert network. If the gating
vector is sparse, we can avoid computing the outputs of experts whose gates are zero. There are multiple 
alternative ways of implementing G(x) [6, 15, 35], but a simple and performant one is implemented by taking the 
softmax over the Top-K logits of a linear layer [28].\n\nThe chunk describes the architectural details of the 
Mixtral model, specifically the Sparse Mixture of Experts (SMoE) layer that is used in the model.',
        'dense_score': 0.4404994080056008
    },
    {
        'id': 7,
        'text': 'We use G(x) := Softmax(TopK(x Â· Wg)), where (TopK(â ))i := â i if â i is among the top-K 
coordinates of logits â â Rn and (TopK(â ))i := â â otherwise. The value of K â the number of experts used per 
token â is a hyper-paramet

In [96]:
import numpy as np

# Normalize the two types of scores
dense_scores = np.array([doc.get("dense_score", 0) for doc in documents_with_scores])
sparse_scores = np.array([doc.get("sparse_score", 0) for doc in documents_with_scores])

dense_scores_normalized = (dense_scores - np.min(dense_scores)) / (np.max(dense_scores) - np.min(dense_scores))
sparse_scores_normalized = (sparse_scores - np.min(sparse_scores)) / (np.max(sparse_scores) - np.min(sparse_scores))

# Calculate a weighted score with alpha of 0.2 to the sparse score
alpha = 0.2
weighted_scores = (1 - alpha) * dense_scores_normalized + alpha * sparse_scores_normalized

# Pick up the top 3 documents with the weighted score
top_docs = sorted(
    zip(
        documents_with_scores, 
        weighted_scores
    ), 
    key=lambda x: x[1], 
    reverse=True
)[:3]



In [95]:
console.print(top_docs)

[
    (
        {
            'id': 15,
            'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length 
Passkey Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 
24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral
has 100% retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input 
sequence. (Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
            'dense_score': 0.6180975806351034,
            'sparse_score': 1.4473938
        },
        0.9371997291627164
    ),
    (
        {
            'id': 4,
            'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
            'dense_score': 0.4972174713599332,
            'sparse_score': 1.1622587
        },
        0.5439267419536705
    ),
    (
        {
            'id': 2,
            'text': 'expertsâ ) to process the token and combine their output additively. This technique increases 
the number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the 
total set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens.
It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
            'dense_score': 0.44239135249907124,
            'sparse_score': 2.1099076
        },
        0.4676420260341326
    )
]

In [97]:
# define a variable to hold the search results
search_results = [doc[0]['text'] for doc in top_docs]

In [98]:
# Now time to connect to the large language model
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are chatbot, an research expert. Your top priority is to help guide users to understand reserach papers."},
        {"role": "user", "content": query},
        {"role": "assistant", "content": str(search_results)}
    ]
)

response_text = Text(completion.choices[0].message.content)

In [99]:
response_text

The context size of Mixtral is 32k tokens. This means that the model can process and utilize information from up to
32,000 tokens in its input data to make predictions and generate output.